#Unsupervised Learning
Implementing unsupervised learning algorithm as described in the given research paper https://arxiv.org/abs/2009.02930



In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from collections import Counter

In [4]:
df = pd.read_csv("SWaT_Dataset_Attack_v0.csv")
print(head)

In [5]:
cols = df.columns
# convert normal and attack innto binary variable
df['Normal/Attack'].replace('Normal', 0, inplace=True)
df['Normal/Attack'].replace('Attack', 1, inplace=True)

In [6]:
df['Normal/Attack'].value_counts()

In [ ]:
df.drop(' Timestamp', axis = 1, inplace=True) 

In [7]:
train_y = df['Normal/Attack']
X_df = df.iloc[:, :-1]

In [8]:
Counter(train_y)

Counter({0: 395298, 1: 54621})

In [9]:
X_df.shape

(449919, 51)

#Robust Principal Component Analysis
Function for RPCA. The approach used is from the paper https://arxiv.org/pdf/0912.3599.pdf

In [13]:
from __future__ import division, print_function

import numpy as np


try:
    # Python 2: 'xrange' is the iterative version
    range = xrange
except NameError:
    # Python 3: 'range' is iterative - no need for 'xrange'
    pass


class R_pca:

    def __init__(self, D, mu=None, lmbda=None):
        self.D = D
        self.S = np.zeros(self.D.shape)
        self.Y = np.zeros(self.D.shape)

        if mu:
            self.mu = mu
        else:
            self.mu = np.prod(self.D.shape) / (4 * np.linalg.norm(self.D, ord=1))

        self.mu_inv = 1 / self.mu

        if lmbda:
            self.lmbda = lmbda
        else:
            self.lmbda = 1 / np.sqrt(np.max(self.D.shape))

    @staticmethod
    def frobenius_norm(M):
        return np.linalg.norm(M, ord='fro')

    @staticmethod
    def shrink(M, tau):
        return np.sign(M) * np.maximum((np.abs(M) - tau), np.zeros(M.shape))

    def svd_threshold(self, M, tau):
        U, S, V = np.linalg.svd(M, full_matrices=False)
        return np.dot(U, np.dot(np.diag(self.shrink(S, tau)), V))

    def fit(self, tol=None, max_iter=1000, iter_print=100):
        iter = 0
        err = np.Inf
        Sk = self.S
        Yk = self.Y
        Lk = np.zeros(self.D.shape)

        if tol:
            _tol = tol
        else:
            _tol = 1E-7 * self.frobenius_norm(self.D)

        #this loop implements the principal component pursuit (PCP) algorithm
        #located in the table on page 29 of https://arxiv.org/pdf/0912.3599.pdf
        while (err > _tol) and iter < max_iter:
            Lk = self.svd_threshold(
                self.D - Sk + self.mu_inv * Yk, self.mu_inv)                            #this line implements step 3
            Sk = self.shrink(
                self.D - Lk + (self.mu_inv * Yk), self.mu_inv * self.lmbda)             #this line implements step 4
            Yk = Yk + self.mu * (self.D - Lk - Sk)                                      #this line implements step 5
            err = self.frobenius_norm(self.D - Lk - Sk)
            iter += 1
            if (iter % iter_print) == 0 or iter == 1 or iter > max_iter or err <= _tol:
                print('iteration: {0}, error: {1}'.format(iter, err))

        self.L = Lk
        self.S = Sk
        return Lk, Sk



In [ ]:
rpca = R_pca(X_df)
L, S = rpca.fit(max_iter=2500, iter_print=100)

iteration: 1, error: 204.87398230969433
iteration: 100, error: 4.4069446695116845
iteration: 200, error: 2.454558499167473
iteration: 300, error: 1.38108625652243
iteration: 400, error: 1.0897614919917433
iteration: 500, error: 0.8263113661332471
iteration: 600, error: 0.663239193398805
iteration: 700, error: 0.6684110349925311
iteration: 800, error: 0.5125580228057208
iteration: 900, error: 0.5041732790895694
iteration: 1000, error: 0.4630132919911478
iteration: 1100, error: 0.42947625480114604
iteration: 1200, error: 0.38970317472346605
iteration: 1300, error: 0.39871112698171923
iteration: 1400, error: 0.3556460150549144
iteration: 1500, error: 0.3452249318968497
iteration: 1600, error: 0.35634241564469127
iteration: 1700, error: 0.3452141982039736
iteration: 1800, error: 0.3330211313131723
iteration: 1900, error: 0.34358468132297
iteration: 2000, error: 0.342936556294922
iteration: 2100, error: 0.34845534127559147
iteration: 2200, error: 0.3589305149826752


In [ ]:
# the noise matrix
S

In [ ]:
S.to_csv('S_matrix.csv') 

In [ ]:
# the low rank matrix of M
L

In [ ]:
save('L_matrix.npy', L)

In [ ]:
L.shape

Function to get the orthogonal basis vector of the low rank matrix L

In [ ]:

def gram_schmidt(A):
    """Orthogonalize a set of vectors stored as the columns of matrix A."""
    # Get the number of vectors.
    n = A.shape[1]
    for j in range(n):
        # To orthogonalize the vector in column j with respect to the
        # previous vectors, subtract from it its projection onto
        # each of the previous vectors.
        for k in range(j):
            A[:, j] -= np.dot(A[:, k], A[:, j]) * A[:, k]
        A[:, j] = A[:, j] / np.linalg.norm(A[:, j])
    return A

In [ ]:
A = gram_schmidt(L)

In [ ]:
#set of orthogonal basis vector spaning the space S of the normal state of the system
A

In [ ]:
save('A_matrix.npy', A)

Projecting the dataset on to the lowdimension subspace S

In [ ]:
Proj_x = A@A.T@X_df

Calculating the Geometric median of the the projected points as they will capture the behaviour of the stable system 

In [ ]:
from scipy.spatial.distance import cdist, euclidean

def geometric_median(X, eps=1e-5):
    y = np.mean(X, 0)

    while True:
        D = cdist(X, [y])
        nonzeros = (D != 0)[:, 0]

        Dinv = 1 / D[nonzeros]
        Dinvs = np.sum(Dinv)
        W = Dinv / Dinvs
        T = np.sum(W * X[nonzeros], 0)

        num_zeros = len(X) - np.sum(nonzeros)
        if num_zeros == 0:
            y1 = T
        elif num_zeros == len(X):
            return y
        else:
            R = (T - y) * Dinvs
            r = np.linalg.norm(R)
            rinv = 0 if r == 0 else num_zeros/r
            y1 = max(0, 1-rinv)*T + min(1, rinv)*y

        if euclidean(y, y1) < eps:
            return y1

        y = y1

In [ ]:
m = geometric_median(Proj_x)

In [ ]:
m.shape

In [ ]:
save('m_vector.npy', m)

In [ ]:
L[0].shape

Calculating the Threshold value to detect anomaly

In [ ]:
min_float = float('-inf')
theta = min_float
for i in range(L.shape[0]):
    dist = np.linalg.norm(m-L[i])
    theta = max(theta,dist)

Function to test new data point

In [ ]:
def classification_onepoint(x):
    proj = A@A.T@x
    score = np.linalg.norm(m-proj) #calculating distance from the geometric median

    if score > theta: #comparing with the threshold value
        return 0
    else:
        return 1